# Timoshenko Beam Fixed-end Reactions

### Differential equations:
$$
M = EI\frac{d\theta}{dx}
$$
$$
V = GAK_s\left(\theta + \frac{dw}{dx}\right)
$$

### Integral form:
$$
\theta = \frac{1}{EI}\int{M}dx \tag{1}
$$
$$
w = \int{\left(\frac{S L^2}{12EI}V - \theta\right)}dx \tag{2}
$$

### Boundary conditions:
$$
\left.\frac{dw}{dx}\right|_{x=0}= 0 \tag{3}
$$
$$
w(0) = 0 \tag{4}
$$
$$
\left.\frac{dw}{dx}\right|_{x=L} = w(L) = 0 \tag{5}
$$
$$
w(L) = 0 \tag{6}
$$

where:\
 $w =$ transverse deflection\
 $\theta =$ rotation about transverse axis\
 $L =$ beam length\
 $K_s =$ shape correction coefficient\
 $E =$ elastic modulus\
 $G =$ shear modulus\
 $A =$ cross-sectional area\
 $I =$ moment of inertia about transverse axis\
 $S = \frac{12EI}{GAK_sL^2}$

let:\
 $R_a =$ fixed-end vertical reaction at $x=0$\
 $R_b =$ fixed-end vertical reaction at $x=L$\
 $M_a =$ fixed-end moment at $x=0$\
 $M_b =$ fixed-end moment at $x=L$

In [1]:
import sympy
from sympy import *
from IPython.display import display, Latex
x, R_a, M_a, EI, L, S = symbols('x R_a M_a EI L S')

## Concentrated Load

Let:\
 $P =$ concentrated load\
 $a =$ location from left end


In [25]:
a, P = symbols('a P')

# Shear and moment equations

# 0 <= x <= a
V1 = R_a
M1 = R_a*x - M_a

# a <= x <= L
V2 = R_a + P
M2 = R_a*x - M_a + P*(x - a)

\
Compatibility equation of $\theta$ at $x=a$

In [26]:
# Integration constant
C = symbols('C')

# Integrate (1) for 0 <= x <= a with boundary condition (3)
theta1 = 1/EI*(M1.integrate(x) + C)
C1 = solve(theta1.subs(x, 0), C)[0]
theta1 = theta1.subs(C, C1)

# Integrate (1) for a <= x <= L with boundary condition (5)
theta2 = 1/EI*(M2.integrate(x) + C)
C1 = solve(theta2.subs(x, L), C)[0]
theta2 = theta2.subs(C, C1)

# Compatibility of theta at x = a 
eq1 = (theta1 - theta2).subs(x, a).simplify()
display(Latex(r'\begin{equation}' + latex(eq1) + r'=0 \tag{7}\end{equation}'))

<IPython.core.display.Latex object>

\
Compatibility equation of $w$ at $x=a$

In [27]:
# Integrate (2) for 0 <= x <= a with boundary condition (4)
w1 = (V1*S*L*L/12/EI - theta1).integrate(x) + C
C1 = solve(w1.subs(x, 0), C)[0]
w1 = w1.subs(C, C1)

# Integrate (2) for a <= x <= L with boundary condition (6)
w2 = (V2*S*L*L/12/EI - theta2).integrate(x) + C
C1 = solve(w2.subs(x, L), C)[0]
w2 = w2.subs(C, C1)

# Compatibility of w at x = a 
eq2 = (w1 - w2).subs(x, a).simplify()
display(Latex(r'\begin{equation}' + latex(eq2) + r'=0 \tag{8}\end{equation}'))

<IPython.core.display.Latex object>

Fixed-end moments

In [28]:
# Solve (7) and (8) for R_a and M_a
soln_point = solve((eq1, eq2), (R_a, M_a))

# Right end shear moments
R_b = -P - R_a
M_b = R_a*L - M_a + P*(L - a)

In [29]:
display(Latex(
    r'\begin{align}' + 
        r'R_a&=' + latex(soln_point[R_a].factor())  +    
        r'\\ R_b&=' + latex(R_b.simplify()) +    
        r'\\ M_a&=' + latex(soln_point[M_a].factor()) + 
        r'\\ M_b&=' + latex(M_b.simplify()) +
    r'\end{align}'
))

<IPython.core.display.Latex object>

The equations above assume Z-axis bending. For Y-axis bending, $M_a$ and $M_b$ are negative of the above.

#### Concentrated Load with $S=0$ Yields the Formula for Euler-Bernoulli Beam

In [7]:
display(Latex(
    r'\begin{align}' + 
        r'R_a&=' + latex(soln_point[R_a].subs(S, 0).factor())  +    
        r'\\ R_b&=' + latex(R_b.subs(S, 0).simplify()) +    
        r'\\ M_a&=' + latex(soln_point[M_a].subs(S, 0).factor()) + 
        r'\\ M_b&=' + latex(M_b.subs(S, 0).simplify()) +
    r'\end{align}'
))

<IPython.core.display.Latex object>

## Distributed Load

Let:\
 $q1 =$ start value distributed load\
 $q2 =$ end value distributed load\
 $x1 =$ start location of distributed\
 $x2 =$ end location of distributed

In [8]:
q1, q2, x1, x2 = symbols('q1 q2 x1 x2')

# Shear and moment equations for 0 <= x <= x1
V1 = R_a
M1 = R_a*x - M_a

# Uniform load qx within x1 <= x <= x2
qx = q1 + (x - x1)*(q2 - q1)/(x2 - x1)

# Resultants of triangular partitions of trapezoidal load
r1 = 0.5*q1*(x - x1)
r2 = 0.5*qx*(x - x1)

# Shear and moment equations for x1 <= x <= x2
V2 = R_a + r1 + r2
M2 = R_a*x - M_a + r1*(x - x1)*2/3 + r2*(x - x1)/3

# Shear and moment equations for x2 <= x <= L
V3 = R_a + r1.subs(x, x2) + r2.subs(x, x2)
M3 = R_a*x - M_a + r1.subs(x, x2)*((x2-x1)*2/3 + (x-x2)) + r2.subs(x, x2)*((x2-x1)/3 + (x-x2))

\
Integrate $(1)$

In [9]:
# Integrate (1) for 0 <= x <= x1 with boundary condition (3)
theta1 = 1/EI*(M1.integrate(x) + C)
C1 = solve(theta1.subs(x, 0), C)[0]
theta1 = theta1.subs(C, C1)

# Theta at x1
theta_x1 = theta1.subs(x, x1)

# Integrate (1) for x1 <= x <= x2 with boundary condition theta = theta_x1 at x = x1
theta2 = 1/EI*(M2.integrate(x) + C)
C1 = solve(theta2.subs(x, x1) - theta_x1, C)[0]
theta2 = theta2.subs(C, C1)

# Theta at x2
theta_x2 = theta2.subs(x, x2)

# Integrate (1) for x2 <= x <= L with boundary condition theta = theta_x2 at x = x2
theta3 = 1/EI*(M3.integrate(x) + C)
C1 = solve(theta3.subs(x, x2) - theta_x2, C)[0]
theta3 = theta3.subs(C, C1)

# Impose boundary condition (5) to theta3 
eq1 = theta3.subs(x, L).simplify()
display(Latex(r'\begin{equation}' + latex(eq1) + r'=0 \tag{9}\end{equation}'))

<IPython.core.display.Latex object>

\
Integrate $(2)$

In [10]:
# Integrate (2) for 0 <= x <= x1 with boundary condition (4)
w1 = (V1*S*L*L/12/EI - theta1).integrate(x) + C
C1 = solve(w1.subs(x, 0), C)[0]
w1 = w1.subs(C, C1)

# w at x1
w_x1 = w1.subs(x, x1)

# Integrate (2) for x1 <= x <= x2 with boundary condition w = w_x1 at x = x1
w2 = (V2*S*L*L/12/EI - theta2).integrate(x) + C
C1 = solve(w2.subs(x, x1) - w_x1, C)[0]
w2 = w2.subs(C, C1)

# w at x2
w_x2 = w2.subs(x, x2)

# Integrate (2) for x2 <= x <= L with boundary condition w = w_x2 at x = x2
w3 = (V3*S*L*L/12/EI - theta3).integrate(x) + C
C1 = solve(w3.subs(x, x2) - w_x2, C)[0]
w3 = w3.subs(C, C1)

# Impose boundary condition (6) to w3
eq2 = w3.subs(x, L).simplify()
display(Latex(r'\begin{equation}' + latex(eq2) + r'=0 \tag{10}\end{equation}'))

<IPython.core.display.Latex object>

\
Solving end moments from $(9)$ and $(10)$

In [11]:
# Solve (9) and (10) for R_a and M_a
soln = solve((eq1, eq2), (R_a, M_a))

# Right end shear moments
R_b = -r1.subs(x, x2) - r2.subs(x, x2) - R_a
M_b = R_a*L - M_a + r1.subs(x, x2)*((x2-x1)*2/3 + (L-x2)) + r2.subs(x, x2)*((x2-x1)/3 + (L-x2))

In [12]:
display(Latex(
    r'\begin{align}' + 
        r'R_a&=' + latex(soln[R_a].simplify())  +    
        r'\\ R_b&=' + latex(R_b.simplify()) +    
        r'\\ M_a&=' + latex(soln[M_a].simplify()) + 
        r'\\ M_b&=' + latex(M_b.simplify()) +
    r'\end{align}'
))

<IPython.core.display.Latex object>

The equations above assume Z-axis bending. For Y-axis bending, $M_a$ and $M_b$ are negative of the above.

#### Distributed Load with $S=0$ Yields the Formula for Euler-Bernoulli Beam

In [13]:
# Formula currently implimented for Euler-Bernoulli Beam (S=0)
display(Latex(
    r'\begin{align}' + 
        r'R_a&=' + latex((soln[R_a].subs(S, 0)/(x1-x2) * 20).simplify() / 20 * (x1-x2))  +    
        r'\\ R_b&=' + latex((R_b.subs({S:0, R_a:soln[R_a]})/(x1-x2) * 20).simplify() / 20 * (x1-x2)) +    
        r'\\ M_a&=' + latex((soln[M_a].subs(S, 0)/(x1-x2) * 60).simplify() / 60 * (x1-x2)) + 
        r'\\ M_b&=' + latex((M_b.subs({S:0, M_a:soln[M_a], R_a:soln[R_a]})/(x1-x2) * 60).simplify() / 60 * (x1-x2)) +
    r'\end{align}'
))

<IPython.core.display.Latex object>

### Concentrated Moment

Let:\
 $M =$ concentrated moment\
 $a =$ location from left end

In [14]:
M = symbols('M')

# Shear and moment equations

# 0 <= x <= a
V1 = R_a
M1 = R_a*x - M_a

# a <= x <= L
V2 = R_a
M2 = R_a*x - M_a - M

In [15]:
# Integrate (1) for 0 <= x <= a with boundary condition (3)
theta1 = 1/EI*(M1.integrate(x) + C)
C1 = solve(theta1.subs(x, 0), C)[0]
theta1 = theta1.subs(C, C1)

# Integrate (1) for a <= x <= L with boundary condition (5)
theta2 = 1/EI*(M2.integrate(x) + C)
C1 = solve(theta2.subs(x, L), C)[0]
theta2 = theta2.subs(C, C1)

# Compatibility of theta at x = a 
eq1 = (theta1 - theta2).subs(x, a).simplify()
display(Latex(r'\begin{equation}' + latex(eq1) + r'=0 \tag{7}\end{equation}'))

<IPython.core.display.Latex object>

In [16]:
# Integrate (2) for 0 <= x <= a with boundary condition (4)
w1 = (V1*S*L*L/12/EI - theta1).integrate(x) + C
C1 = solve(w1.subs(x, 0), C)[0]
w1 = w1.subs(C, C1)

# Integrate (2) for a <= x <= L with boundary condition (6)
w2 = (V2*S*L*L/12/EI - theta2).integrate(x) + C
C1 = solve(w2.subs(x, L), C)[0]
w2 = w2.subs(C, C1)

# Compatibility of w at x = a 
eq2 = (w1 - w2).subs(x, a).simplify()
display(Latex(r'\begin{equation}' + latex(eq2) + r'=0 \tag{8}\end{equation}'))

<IPython.core.display.Latex object>

In [17]:
# Solve (7) and (8) for R_a and M_a
soln_point = solve((eq1, eq2), (R_a, M_a))

# Right end shear moments
R_b = -R_a
M_b = R_a*L - M_a - M

In [18]:
display(Latex(
    r'\begin{align}' + 
        r'R_a&=' + latex(soln_point[R_a].factor())  +    
        r'\\ R_b&=' + latex(R_b.subs({R_a: soln_point[R_a]}).simplify()) +    
        r'\\ M_a&=' + latex(soln_point[M_a].factor()) + 
        r'\\ M_b&=' + latex(M_b.subs({R_a: soln_point[R_a], M_a: soln_point[M_a]}).simplify()) +
    r'\end{align}'
))

<IPython.core.display.Latex object>

The equations above assume Z-axis bending. For Y-axis bending, $R_a$ and $R_b$ are negative of the above.

#### Concentrated moment with $S=0$ Yields the Formula for Euler-Bernoulli Beam

In [19]:
display(Latex(
    r'\begin{align}' + 
        r'R_a&=' + latex(soln_point[R_a].subs(S, 0).factor())  +    
        r'\\ R_b&=' + latex(R_b.subs({R_a: soln_point[R_a], S:0}).simplify()) +    
        r'\\ M_a&=' + latex(soln_point[M_a].subs(S, 0).factor()) + 
        r'\\ M_b&=' + latex(M_b.subs({R_a: soln_point[R_a], M_a: soln_point[M_a], S:0}).simplify()) +
    r'\end{align}'
))

<IPython.core.display.Latex object>